In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats #The SciPy stats module

from functions import chunks #for splitting number of stocks for batch API calls
from secrets import IEX_CLOUD_API_TOKEN
from statistics import mean

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# set it up!

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
#########
rv_columns = [
    'Ticker',
    'Price',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
      # include more try expect statements!
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )
rv_dataframe.head()

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.250,51.59,N/A,8.72,N/A,7.59,N/A,33.505927,N/A,14.135118,N/A,N/A
1,AAL,23.690,-1.59,N/A,-1.89,N/A,1.2,N/A,-3.801511,N/A,3.090557,N/A,N/A
2,AAP,190.600,27.32,N/A,3.55,N/A,1.26,N/A,12.752013,N/A,2.813367,N/A,N/A
3,AAPL,132.080,29.07,N/A,31.27,N/A,6.57,N/A,21.184315,N/A,16.706604,N/A,N/A
4,ABBV,115.651,40,N/A,15.27,N/A,4.18,N/A,12.998434,N/A,8.437714,N/A,N/A


In [4]:
# Dealing with missing values
rv_dataframe.dropna(axis=0, inplace=True, subset=["Ticker", "Price", 'Price-to-Earnings Ratio', 'Price-to-Book Ratio',
                                                 'Price-to-Sales Ratio','EV/EBITDA','EV/GP'])

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [6]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
 



for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = 1-mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,138.250,51.59,0.825103,8.72,0.772634,7.59,0.787037,33.505927,0.876543,14.135118,0.781893,0.191358
1,AAL,23.690,-1.59,0.135802,-1.89,0.0514403,1.2,0.118313,-3.801511,0.0411523,3.090557,0.131687,0.904321
2,AAP,190.600,27.32,0.516461,3.55,0.467078,1.26,0.126543,12.752013,0.337449,2.813367,0.121399,0.686214
3,AAPL,132.080,29.07,0.54321,31.27,0.95679,6.57,0.740741,21.184315,0.654321,16.706604,0.855967,0.249794
4,ABBV,115.651,40,0.736626,15.27,0.874486,4.18,0.576132,12.998434,0.358025,8.437714,0.504115,0.390123
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.780,32.36,0.606996,-4.62,0.0493827,6.24,0.730453,23.900678,0.72428,10.315869,0.619342,0.453909
501,ZBH,173.686,61.9,0.868313,2.94,0.382716,4.95,0.644033,29.120433,0.839506,8.703959,0.520576,0.348971
502,ZBRA,513.050,42.67,0.769547,11.67,0.829218,5.72,0.6893,31.049741,0.860082,12.244969,0.720165,0.226337
503,ZION,58.530,6.85,0.144033,1.33,0.110082,2.84,0.400206,5.772195,0.0617284,2.764117,0.117284,0.833333


In [7]:
rv_dataframe.sort_values(by = 'RV Score', ascending = False,  inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,30.090,7.96,0.148148,0.5988,0.0534979,0.4713,0.0226337,4.098794,0.0514403,0.476557,0.0102881,0.942798
1,AIV,7.059,-77.42,0.0288066,2.04,0.268519,-31.81,0.00205761,-42.945879,0.00823045,-53.712235,0.00617284,0.937243
2,L,58.800,-433.16,0.00411523,0.9163,0.0679012,1.13,0.103909,8.638307,0.13786,1.086305,0.0308642,0.93107
3,AIG,52.157,-11.93,0.0946502,0.7187,0.0576132,1.07,0.0967078,6.518779,0.0679012,1.053550,0.0288066,0.930864
4,MCK,198.000,-7.18,0.111111,-1543.3,0.00411523,0.1373,0.00823045,7.787603,0.111111,2.708403,0.111111,0.930864
5,HFC,34.060,-36.6,0.0555556,1.11,0.0802469,0.4984,0.0246914,8.369207,0.123457,2.841935,0.125514,0.918107
6,FTI,8.300,11.24,0.193416,1,0.0720165,0.3188,0.0123457,3.572466,0.0493827,3.395055,0.150206,0.904527
7,AAL,23.690,-1.59,0.135802,-1.89,0.0514403,1.2,0.118313,-3.801511,0.0411523,3.090557,0.131687,0.904321
8,ABC,120.530,-6.32,0.119342,-90.08,0.00823045,0.124,0.00617284,9.099945,0.158436,4.394445,0.216049,0.898354
9,VIAC,41.470,9,0.158436,1.43,0.133745,1.04,0.090535,7.250061,0.0864198,1.457978,0.0514403,0.895885
